In [1]:
# Utiles
    # Lib: https://github.com/sammchardy/python-binance
    # Binance doc : https://github.com/binance-exchange/binance-official-api-docs/blob/master/rest-api.md
    
from binance.client import Client
api_key = '8te7KGDgi6ayc8slY46KbWztB7yuIZ533fCrLWeqARAxBKAuQiMyyfIM4mfDrSlk'
api_secret = 'ykwjZzkYF4Bvh0T7L1OokHfHTnVjKJ4OIvaEXkV97NisgI4JwDW7jPd3pzeVVpHh'

client = Client(api_key, api_secret)

In [2]:
# Others

# For a trading pair status = 'TRADING'
client.get_symbol_info('BNBBTC')

# Get server time
client.get_server_time()

{'serverTime': 1545126806820}

In [2]:
# ============= Check before trading =============

# ping
client.ping()

# Check system status
status = client.get_system_status()
if (status['msg'] == 'normal') and (status['status'] == 0):
    print('ok')
    
# Check account status
status_client = client.get_account_status()
if (status_client['msg'] == 'Normal') and (status_client['success'] == True):
    print('ok')
    
# Check account status - trading allowed
info_client = client.get_account()
if info_client['canTrade'] == True:
    print('ok')

# Exchange info to verify tradingpair exists / trading
exchange_info = client.get_exchange_info()
for symbol in exchange_info['symbols']:
    # si symbol['symbol'] dans trading pairs
    # symbol['status'] doit être égal à TRADING sinon msg erreur + trade pas dessus sur cette itération



{'msg': 'Normal', 'success': True}

In [54]:
for key, value in symbol.items() :
    print (key)

symbol
status
baseAsset
baseAssetPrecision
quoteAsset
quotePrecision
orderTypes
icebergAllowed
filters


In [20]:
# ============== get_price() ==============
    # => get_portfolio_value() => see up, retrieve ticker
    # => create_order() :
        # Side=Buy: get_order_book() + asks (mettre un peu plus haut ?)
        # Side=Sell: get_order_book + bids (mettre un peu plus bas ?)
# get market depth (see what should be approx the price, use to determine limit to put to be sure to buy / sell directly)
depth = client.get_order_book(symbol='ETHUSDT')

# bid: maximum price buyer willing to pay
# ask: minimum price seller willing to receive
float(depth['bids'][0][0])

85.91

In [64]:
'asks' in depth

True

In [12]:
# SHOULD NOT BE THE WAY I USE
# get all symbol prices
prices = client.get_all_tickers()

# get one symbol price
prices_dict = { value["symbol"]: value["price"] for value in prices }
prices_dict['ETHUSDT']

In [3]:
# ============== get_available_amount_crypto()  ==============
# balance for an asset
client.get_asset_balance(asset='VET')

{'asset': 'VET', 'free': '70648.90000000', 'locked': '0.00000000'}

In [31]:
# ============== create_order() ==============
def format_amount_order(amount):
    precision = 5
    amt_str = "{:0.0{}f}".format(amount, precision)
    return amt_str

# limit buy
order = client.order_limit_buy(
    symbol='BNBBTC',
    quantity=1000,
    price=format_amount_order(0.00001)) # Use Ask + n%

# limit sell
order = client.order_limit_sell(
    symbol='BNBBTC',
    quantity=1000,
    price=format_amount_order(0.00001)) # Use Bid - n%

In [12]:
order = client.order_limit_buy(
    symbol='BNBBTC',
    quantity=1000,
    price=format_amount_order(0.001)) # Use Ask + n%

BinanceAPIException: APIError(code=-2010): Account has insufficient balance for requested action.

In [4]:
# ============== cancel_open_orders()  ==============
orders = client.get_open_orders()
# boucle:
# result = client.cancel_order(
#     symbol='BNBBTC',
#     orderId='orderId')
# Attention, sell order <> buy order ! cancel buy ok, cancel sell ? risqué..